# Hit classification

In this notebook we will try to use Keras and XGBoost in order to distinguish between _true_ conversion electron hits and _fake_ conversion electron hits.

In [1]:
import uproot 
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, ReLU
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from xgboost import XGBClassifier

2022-12-23 17:32:34.657232: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


First of all we use [uproot](https://uproot.readthedocs.io/en/latest/), a library to reading ROOT files in pure Python and NumPy, to open the `trkana` tree in the `TAKK` folder of the `KKSM01.root` file. We only need certain branches, so we apply a filter to read only `de`, `detsh`, `detshmc`, and `demc`. We then apply a mask to our tree to select only good fits. 

In [2]:
input_dataset = np.empty
temp = np.empty
signal = np.empty
backgnd = np.empty
filelist = os.environ['KKTrainData']
print("Using files in " + filelist)
files = open(filelist, 'r')
for filename in files:
    print("Processing: " + filename)    
    with uproot.open(filename) as file:
        trkana = file["TAKK"]["trkana"].arrays(filter_name="/de|detsh|detshmc|demc/i")
        trkana = trkana[(trkana['de.goodfit']==1)&(trkana['de.status']>0)&(trkana['demc.proc']==167)]
    udt = ak.concatenate(trkana['detsh.udt']).to_numpy()
    udoca = ak.concatenate(trkana['detsh.udoca']).to_numpy()
    rdrift = ak.concatenate(trkana['detsh.cdrift']).to_numpy()
    tottdrift = ak.concatenate(trkana['detsh.tottdrift']).to_numpy()
    edep = ak.concatenate(trkana['detsh.edep']).to_numpy()
    udocavar = ak.concatenate(trkana['detsh.udocavar']).to_numpy()
    hstate = ak.concatenate(trkana['detsh.state']).to_numpy()
    print("Processed file " + filename + " with %s hits"%udt.shape[0])
    temp = np.vstack((udt,udoca,tottdrift,rdrift,edep,udocavar)).T
    if input_dataset is np.empty:
        input_dataset = temp
    else:
        input_dataset = np.concatenate((input_dataset, temp))
    mcrel = []
    mcambig = []
    mcdist = []
    for i, this_dt in enumerate(trkana['detsh.udt']):
        mcrel.extend(trkana['detshmc.rel._rel'][i][:len(this_dt)])
        mcambig.extend(trkana['detshmc.ambig'][i][:len(this_dt)])
        mcdist.extend(trkana['detshmc.dist'][i][:len(this_dt)])
    mcrel = np.array(mcrel)
    mcambig = np.array(mcambig)
    mcdist = np.array(mcdist)

    sig = (hstate>-2) & (mcrel==0) & (mcambig*udoca>0) & (rdrift-mcdist<0.4)
    bkg = (hstate>-2) & (mcrel==0) & ((mcambig*udoca<0) | (rdrift-mcdist>0.4))
    if signal is np.empty:
        signal = sig
        backgnd = bkg
    else:
        signal = np.concatenate((signal,sig))
        backgnd = np.concatenate((backgnd,bkg))    
nhits=len(input_dataset)
print("Total dataset %s hits"%nhits)

Using files in /Users/brownd/data/42082053/files.txt
Processing: /Users/brownd/data/42082053/nts.brownd.KKSeed.KKSNB.001210_00000005.root

Processed file /Users/brownd/data/42082053/nts.brownd.KKSeed.KKSNB.001210_00000005.root
 with 580548 hits
Processing: /Users/brownd/data/42082053/nts.brownd.KKSeed.KKSNB.001210_00000008.root

Processed file /Users/brownd/data/42082053/nts.brownd.KKSeed.KKSNB.001210_00000008.root
 with 571683 hits
Processing: /Users/brownd/data/42082053/nts.brownd.KKSeed.KKSNB.001210_00000024.root

Processed file /Users/brownd/data/42082053/nts.brownd.KKSeed.KKSNB.001210_00000024.root
 with 575947 hits
Total dataset 1728178 hits


Here we then assign a label to each hit as _signal_ or _background_, depending on the Monte Carlo truth information. Since the dimension of  `detshmc.rel._rel` is not guaranteed to be the same as the dimension of `detsh` we need to loop over all the entries.

In our problem we have a different number of signal and background entries in our input dataset. There are several techniques avaialable for _unbalanced_ datasets. Here we are using the most naive one, which is just using $\min(N_{sig}, N_{bkg})$ events. Then, we divide our input into the _training_, _validation_, and _test_ datasets.

In [3]:
min_len = min(len(input_dataset[signal]), len(input_dataset[backgnd]))
bsize=32
# I need to double the batch_size when truncating as we divide the sample in half later for training
tsize=2*bsize
min_len = min_len - min_len%tsize
print("Training on %s matched hits"%min_len)
signal_dataset = input_dataset[signal][:min_len]
bkg_dataset = input_dataset[backgnd][:min_len]

balanced_input = np.concatenate((signal_dataset, bkg_dataset))
y_balanced_input = np.concatenate((np.ones(signal_dataset.shape[0]), np.zeros(bkg_dataset.shape[0])))

n_variables = balanced_input.shape[1]

x_ce_train, x_ce_test, y_ce_train, y_ce_test = train_test_split(balanced_input, y_balanced_input, test_size=0.5, random_state=42)
x_ce_test, x_ce_valid, y_ce_test, y_ce_valid = train_test_split(x_ce_test, y_ce_test, test_size=0.5, random_state=42)

Training on 242816 matched hits


## Create and train a multi-layer perceptron

Here we create a _multi-layer perceptron_ (MLP) model which consists of 3 fully-connected (or _dense_) layers, each one followed by a _dropout_ layer, which helps to avoid overfitting. The model is trained using the [Adam](https://arxiv.org/abs/1412.6980) optimizer and trained for 50 epochs or until the validation loss doesn't improve for 5 epochs (`early_stop`). 

In [ ]:
lay0=Input(shape=(n_variables,),batch_size=1)
lay1=Dense(2*n_variables, activation='relu')(lay0)
lay2=Dense(2*n_variables, activation='relu')(lay1)
lay3=Dense(2*n_variables, activation='relu')(lay2)
lay4=Dense(1,activation='sigmoid')(lay3)
output_model=Model(inputs=lay0,outputs=lay4)

opt = Adam(learning_rate=1e-3)
input=Input(shape=(n_variables,),batch_size=bsize)
x=Dense(2*n_variables, activation='relu')(input)
x=Dense(2*n_variables, activation='relu')(x)
x=Dense(2*n_variables, activation='relu')(x)
output=Dense(1,activation='sigmoid')(x)
model_ce=Model(inputs=input,outputs=output)
model_ce.compile(loss='binary_crossentropy',metrics='accuracy',optimizer=opt)
early_stop = EarlyStopping(monitor='val_loss', patience=10, min_delta=1e-5, restore_best_weights=True)
history_ce = model_ce.fit(x_ce_train, y_ce_train,
                          batch_size=bsize,
                          epochs=200,
                          verbose=1,
                          validation_data=(x_ce_valid, y_ce_valid),
                          callbacks=[early_stop]
                         )

Epoch 1/200


2022-12-23 17:34:07.654545: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


7588/7588 [==============================] - 8s 1ms/step - loss: 0.5456 - accuracy: 0.7436 - val_loss: 0.5057 - val_accuracy: 0.7683
Epoch 2/200
7588/7588 [==============================] - 8s 1ms/step - loss: 0.4950 - accuracy: 0.7705 - val_loss: 0.4957 - val_accuracy: 0.7738
Epoch 3/200
7588/7588 [==============================] - 8s 1ms/step - loss: 0.4892 - accuracy: 0.7738 - val_loss: 0.4885 - val_accuracy: 0.7762
Epoch 4/200
7588/7588 [==============================] - 8s 1ms/step - loss: 0.4859 - accuracy: 0.7755 - val_loss: 0.4896 - val_accuracy: 0.7751
Epoch 5/200
7588/7588 [==============================] - 8s 1ms/step - loss: 0.4840 - accuracy: 0.7769 - val_loss: 0.4860 - val_accuracy: 0.7763
Epoch 6/200
7588/7588 [==============================] - 9s 1ms/step - loss: 0.4824 - accuracy: 0.7779 - val_loss: 0.4862 - val_accuracy: 0.7780
Epoch 7/200
7588/7588 [==============================] - 8s 1ms/step - loss: 0.4818 - accuracy: 0.7781 - val_loss: 0.4889 - val_accuracy: 0.77

## Create and train a Boosted Decision Tree
Here, instead of using a MLP, we use a [_Gradient Boosted Decision Tree_](https://xgboost.readthedocs.io/en/stable/) (BDT) to distinguish between signal (true CE hits) and background (fake CE hits). We use the defualt hyperparameters.

In [ ]:
model_xgboost = XGBClassifier()
model_xgboost.fit(x_ce_train, y_ce_train)

Here we can finally apply our two models (the MLP and the BDT) to our test datasets and create the corresponding ROC curves.

In [ ]:
prediction_ce = model_ce.predict(x_ce_test).ravel()
fpr_ce, tpr_ce, th_ce = roc_curve(y_ce_test,  prediction_ce)
auc_ce = roc_auc_score(y_ce_test, prediction_ce)

In [ ]:
prediction_xgboost = model_xgboost.predict_proba(x_ce_test)[:,1]
fpr_xgboost, tpr_xgboost, th_xgboost = roc_curve(y_ce_test,  prediction_xgboost)
auc_xgboost = roc_auc_score(y_ce_test, prediction_xgboost)

The plot of the ROC curves clearly shows that the BDT outperforms the MLP. In principle, however, it should be possible to improve the MLP performances by optimizing the hyperparameters (learning rate, hidden layers, activation functions, etc.).

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(tpr_ce,1-fpr_ce,label=f'MLP AUC: {auc_ce:.3f}')
ax.plot(tpr_xgboost,1-fpr_xgboost,label=f'BDT AUC: {auc_xgboost:.3f}')

ax.legend()
ax.set_aspect("equal")
ax.set_xlabel("Signal efficiency")
ax.set_ylabel("Background rejection")
ax.set_xlim(0,1.05)
ax.set_ylim(0,1.05)
fig.savefig("TrainDrift.pdf")

In [ ]:
output_model.set_weights(model_ce.get_weights())
output_model.summary()
output_model.save("TrainDrift.h5")